In [ ]:
from minijpas_LF_and_puricomp import effective_volume
import pickle
import numpy as np
import matplotlib.pyplot as plt
import emcee
from scipy.optimize import minimize
import corner
from my_functions import schechter

In [ ]:
# My LF

nb_min = 1
nb_max = 25
nbs_list = [[1, 5], [5, 10], [10, 15], [15, 20], [20, 25]]
total_volume = effective_volume(nb_min, nb_max, 'both')

for i, [nb1, nb2] in enumerate(nbs_list):
    pathname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew15_ewoth400_nb'
    filename = f'{pathname}/LFs.pkl'
    with open(filename, 'rb') as file:
        this_LF_dict = pickle.load(file)

    this_volume = effective_volume(nb1, nb2, 'both')

    # In the first iteration of the loop:
    if i == 0:
        LF_bins = this_LF_dict['LF_bins']
        bin_width = np.diff(LF_bins)[0]

        this_LF_raw = this_LF_dict['LF_total'] * this_volume * bin_width
        LF_raw = this_LF_raw / bin_width
        LF_uncorr = this_LF_dict['LF_total_raw'] * this_volume

        LF_err = ((np.array(this_LF_dict['LF_total_err']) *
                  this_volume * bin_width) ** 2 - this_LF_raw) ** 0.5
    # Second iter and further
    else:
        this_LF_raw = this_LF_dict['LF_total'] * this_volume * bin_width
        LF_raw += this_LF_raw / bin_width
        LF_err += ((np.array(this_LF_dict['LF_total_err']) *
                  this_volume * bin_width) ** 2 - this_LF_raw) ** 0.5
        LF_uncorr += this_LF_dict['LF_total_raw'] * this_volume

LF_err = np.array(LF_err)
LF_err[~np.isfinite(LF_err)] = 0

LF_dict = {
    'LF_bins': LF_bins,
    'LF_total': LF_raw / total_volume,
    'LF_total_uncorr': LF_uncorr / total_volume,
    'LF_total_err': (LF_err ** 2 + LF_raw * bin_width) ** 0.5 / total_volume / bin_width
}

LF_phi = LF_dict['LF_total']
LF_bin = LF_dict['LF_bins']
LF_yerr_minus = LF_dict['LF_total_err'][0]
LF_yerr_plus = LF_dict['LF_total_err'][1]
LF_xerr = np.ones(LF_dict['LF_total_err'][2].shape) * bin_width * 0.5

In [ ]:
# The fitting surve
def sch_fit(Lx, Phistar, Lstar):
    return schechter(Lx, 10 ** Phistar, 10 ** Lstar, -1.5) * Lx * np.log(10)

In [ ]:
# Log likelihood:
def log_likelihood(theta, x, y, yerr):
    Phistar, Lstar, log_f = theta
    model = sch_fit(x, Phistar, Lstar)
    sigma2 = yerr**2 + model**2 * np.exp(2 * log_f)
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(sigma2))

# Two free parameters: Phistar and Lstar
# Define the log prior

def log_prior(theta, p0):
    Phistar, Lstar, log_f = theta
    Lstar0_min = p0[0][0]
    Lstar0_max = p0[0][1]
    Phistar0_min = p0[1][0]
    Phistar0_max = p0[1][1]
    # We set the prior:
    if (
        (Lstar0_min < Lstar < Lstar0_max)
        and (Phistar0_min < Phistar < Phistar0_max)
        and (-4 < log_f < 4)
    ):
        return 0.
    else:
        return -np.inf

# Define the log probability

def log_probability(theta, Lx, Phi, yerr, p0):
    lp = log_prior(theta, p0)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, Lx, Phi, yerr)

def nll(*args):
    return -log_likelihood(*args)


In [ ]:
### PRIOR PARAMETERS ###
Lstar0 = np.array([-7, -5])
Phistar0 = np.array([43, 45])
########################

p0 = [Lstar0, Phistar0]
initial = [6, 44, 3]

soln = minimize(nll, initial, args=(LF_bins, LF_phi, LF_err[0]))

pos = soln.x + 1e-4 * np.random.randn(32, 3)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_probability, args=(LF_bins, LF_phi, LF_err[0], p0)
)
sampler.run_mcmc(pos, 5000, progress=True);

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
labels = ['Lstar', 'Phistar']

fig = corner.corner(flat_samples, labels=labels)

In [ ]:
flat_samples